### 02-Pre-Processing

In [1]:
#imports
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split, GridSearchCV


In [2]:
#read in Data
data = pd.read_csv('.././data/reddit_data.csv')
data.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (27,67,70,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,crosspost_parent_list,removed_by_category,media,media_embed,secure_media,secure_media_embed,author_cakeday,poll_data,edited,link_flair_css_class
0,[],False,coioteatomico,NaN,[],NaN,text,t2_7w0v95ej,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,PiratusInteruptus,NaN,[],NaN,text,t2_3sett0rm,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Manawisp,NaN,[],Pothos,text,t2_z7j6n,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,youdontknowmemessy,NaN,[],NaN,text,t2_10d88scc,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,mroebuilds,NaN,[],NaN,text,t2_78cpyr3d,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#I don't need everything that I pulled
post_data = data[['title','subreddit']]
post_data

,title,subreddit
0,Can anyone identify this?,plants
1,Brought mine for the winter,plants
2,Monsters sad. What do I do? North TX,plants
3,"Avacado, are these roots? Also how much longer...",plants
4,My first succulent arrangement! 3 parts soil 2...,plants
...,...,...
9995,The judgement,cats
9996,Muffin my latest foster 😻😻😻,cats
9997,NEW VIDEO!! We have a ton of content on IG.. f...,cats
9998,Sleeping Beauty,cats


In [4]:
#Instantiate RegEx tokenizer
tokenizer = RegexpTokenizer('\w+|\$[\d\.]')
#Run tokenizer
tokenized_data = [tokenizer.tokenize(row) for row in data['title']]

In [5]:
#instantiate tokenizer and lemmatizer, and apply to the title column in the dataframe
# function from https://stackoverflow.com/questions/47557563/lemmatization-of-all-pandas-cells
# added if statement to remove stop words from the text_lemmatized lists
#if w.lower() not in stopwords.words('english') removed so that stop words could be used in model

tokenizer = RegexpTokenizer('\w+|\$[\d\.]')
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    row = [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text) ]
    string = ''
    for word in range(len(row)):
        string = string + row[word].lower() + ' '
    return string
post_data['text_lemmatized'] = post_data['title'].apply(lemmatize_text)

<ipython-input-5-f1755c810f5e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_data['text_lemmatized'] = post_data['title'].apply(lemmatize_text)


In [6]:
post_data

,title,subreddit,text_lemmatized
0,Can anyone identify this?,plants,can anyone identify this
1,Brought mine for the winter,plants,brought mine for the winter
2,Monsters sad. What do I do? North TX,plants,monsters sad what do i do north tx
3,"Avacado, are these roots? Also how much longer...",plants,avacado are these root also how much longer un...
4,My first succulent arrangement! 3 parts soil 2...,plants,my first succulent arrangement 3 part soil 2 p...
...,...,...,...
9995,The judgement,cats,the judgement
9996,Muffin my latest foster 😻😻😻,cats,muffin my latest foster
9997,NEW VIDEO!! We have a ton of content on IG.. f...,cats,new video we have a ton of content on ig follo...
9998,Sleeping Beauty,cats,sleeping beauty


In [7]:
#binarize the target column
post_data['subreddit'] = post_data['subreddit'].map({'plants': 0,'cats':1}).astype(int)

<ipython-input-7-bf94fdd679b0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_data['subreddit'] = post_data['subreddit'].map({'plants': 0,'cats':1}).astype(int)


In [8]:
post_data.head()

,title,subreddit,text_lemmatized
0,Can anyone identify this?,0,can anyone identify this
1,Brought mine for the winter,0,brought mine for the winter
2,Monsters sad. What do I do? North TX,0,monsters sad what do i do north tx
3,"Avacado, are these roots? Also how much longer...",0,avacado are these root also how much longer un...
4,My first succulent arrangement! 3 parts soil 2...,0,my first succulent arrangement 3 part soil 2 p...


### Vectorize the text_lemmatized column

In [9]:
#create X and Y and split data
X = post_data['text_lemmatized']
y = post_data['subreddit']
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    stratify=y,
                                                    random_state=51)

In [10]:
#https://stackoverflow.com/questions/35935670/share-variables-between-different-jupyter-notebooks
#storing these variables so I can use them in the next notebook
%store X_train X_test y_train y_test

Stored 'X_train' (Series)
Stored 'X_test' (Series)
Stored 'y_train' (Series)
Stored 'y_test' (Series)
